In [1]:
import re
import os
import pandas as pd
from typing import Hashable
from calendar import month_name

In [2]:
def file_to_str(year:int):
    path =  f'../metadata_text/{year}_metadata.txt'
    try:
        
        with open(path) as p:
        
            contents = p.read()
        return contents
    except:
        print(f"{path} DNE")

## Clean up initial text file

In [3]:
def clean_regex(pluto_text:str):
    single = '\n[\s\S]\n'
    page_num = '\n\d+\n'
    comb = single + '|' + page_num
    
    try:
        return re.sub(comb, '\n', pluto_text)
    except:
        print("exc")
    

In [4]:
def mk_pluto_headers():
    pluto_dd = "PLUTO.? DATA DICTIONARY"

    header_list = []
    for m in list(month_name[1:]):
        header_list.append(pluto_dd + "\s*"+ m + " 200\d")
        header_list.append( m + " 200\d" + "\s*" + pluto_dd)
    
    return '|'.join(header_list)

In [5]:
HEADS = mk_pluto_headers()

In [6]:
def metadata_list(pluto_string:str):
        matches = re.split('Field Name:\s?', pluto_string)
        l = []
        for match in matches:
            if '. . .' not in match and '...' not in match:
                field = re.split('Format:\s?|Data Source:\s?|Description:\s?', match)
                for i in range(0, len(field)):
                    field[i] = re.sub('\s$', '', field[i])
                    field[i] = re.sub('[\s\n]{2,}', ' ', field[i])
                    field[i] = re.sub('\n', '', field[i])
                    
                l.append(field)
        return l

In [7]:
def text_to_df(yr):
    pluto_text = file_to_str(yr)
    txt = clean_regex(pluto_text)
    txt = re.sub(HEADS, '\n', txt)
    spl_text = metadata_list(txt)
    col_names = ['field_name', 'data_type', 'source', 'definition']
    try:
        df = pd.DataFrame(spl_text, columns=col_names)
        df['year'] = yr
        return df
    except:
        print(f"there was an issue with {yr}")
    

In [8]:
# text_to_df(2004)
l = []

for i in range(2004, 2021):
    if i != 2008:
        l.append(text_to_df(i))

In [9]:
meta_04_20=pd.concat(l)

In [10]:
meta_04_20

,field_name,data_type,source,definition,year
0,Two types of Field Names are included - extern...,The Format is divided into two parts:type of f...,The Data Source identifies the city agency and...,The Description includes a brief explanation o...,2004
1,BOROUGH (Borough),Alphanumeric - 2 characters,Department of City Planning - based on data fr...,The borough that the tax lot is located in.Thi...,2004
2,TAX BLOCK (Block),Numeric - 5 digits (99999),Department of Finance - RPAD Master File,The tax block that the tax lot is located in.T...,2004
3,TAX LOT (Lot),Numeric - 4 digits (9999),Department of Finance - RPAD Master File,The number of the tax lot.This field contains ...,2004
4,COMMUNITY DISTRICT (CD),Numeric - 3 digits (999),Department of City Planning - based on data fr...,The community district (CD) or joint interest ...,2004
...,...,...,...,...,...
86,VERSION NUMBER (Version),Alphanumeric – 6 characters PLUTO DATA DICTION...,Department of City Planning,The version number for this release of PLUTO. ...,2020
87,CHANGED BY DCP (DCPEdited),Alphanumeric – 3 characters,Department of City Planning,Flag indicating that City Planning has applied...,2020
88,LATITUDE (Latitude),Numeric,Department of City Planning,The WGS 84 latitude of the latitude/longitude ...,2020
89,LONGITUDE (Longitude),Numeric,Department of City Planning,The WGS 84 longitude of the latitude/longitude...,2020


In [11]:
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../secrets/secret.json'
bq_client = bigquery.Client()


table_id = "pluto-panel.metadata.metadata_2004_2020"

job = bq_client.load_table_from_dataframe(
    meta_04_20, table_id
)
job.result()  

In [ ]:
def normalize(col_name: Hashable) -> str:
    fixes = [(r"[ /:,?()\.-]", "_"), (r"['’]", "")]
    """Perform normalization of column name."""
    result = str(col_name)
    for search, replace in fixes:
        result = re.sub(search, replace, result)
    return result

In [ ]:
def clean_df(df):
    
    
    split_field = d.field_name.str.split('\s\(', expand = True)
    
    d['friendly_name'] = split_field.iloc[:,0]
    d['field_name'] = split_field.iloc[:,1]\
                      .str.replace(')', '', regex = False)
    
    d['field_name'] = df['field_name'].str.lower()
    d['field_name'] = d['field_name'].apply(normalize)
    return d